## 📦 Passo 1: Importar Bibliotecas

**Novidades neste notebook:**
- `RunnablePassthrough`: Permite passar dados através da chain
- `StrOutputParser`: Formata a saída do LLM como string limpa
- Usaremos o operador `|` para criar chains!

In [1]:
import os
import requests
from pathlib import Path
from dotenv import load_dotenv

from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings, OllamaLLM
from langchain_core.prompts import PromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader

# Imports para LCEL (Chains)
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

## ⚙️ Passo 2: Configurar Ambiente

Mesma configuração do notebook anterior.

In [2]:
# Configurações iniciais
OLLAMA_BASE_URL = 'http://localhost:11434'
BASE_DIR = Path(__file__).parent if "__file__" in globals() else Path.cwd()
PDF_DIR = BASE_DIR.parent.parent / "data" / "pdfs"

print(f"📁 Diretório de PDFs: {PDF_DIR}")
print(f"🤖 Ollama URL: {OLLAMA_BASE_URL}")

📁 Diretório de PDFs: e:\01-projetos\11-work\11.34-engenharia-vetorial\data\pdfs
🤖 Ollama URL: http://localhost:11434


## 📄 Passo 3: Carregar Documentos

Carregamos todos os PDFs da pasta `data/pdfs/`.

In [3]:
# Carrega os documentos
documents = []

pdf_paths = list(PDF_DIR.glob("*.pdf"))
print(f"📚 Encontrados {len(pdf_paths)} PDFs\n")

for pdf_path in pdf_paths:
    loader = PyPDFLoader(str(pdf_path))
    docs = loader.load()
    documents.extend(docs)
    print(f"  ✓ {pdf_path.name}: {len(docs)} páginas")

print(f"\n✅ Total de páginas carregadas: {len(documents)}")

incorrect startxref pointer(1)
parsing for Object Streams
incorrect startxref pointer(1)
parsing for Object Streams
incorrect startxref pointer(1)
parsing for Object Streams
incorrect startxref pointer(1)
parsing for Object Streams


📚 Encontrados 4 PDFs

  ✓ api_documentation.pdf: 3 páginas
  ✓ livro_receitas.pdf: 5 páginas
  ✓ manual_futebol.pdf: 4 páginas
  ✓ manual_iphone15.pdf: 3 páginas

✅ Total de páginas carregadas: 15


## ✂️ Passo 4: Dividir em Chunks

**Chunking strategy:**
- `chunk_size=1000`: Chunks de ~1000 caracteres
- `chunk_overlap=200`: Overlap para não perder contexto nas bordas
- `separators`: Prioriza quebras naturais (parágrafos → linhas → espaços)

In [4]:
# Chunking
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    separators=["\n\n", "\n", " ", ""]
)

chunks = text_splitter.split_documents(documents=documents)

print(f"✂️  Total de chunks criados: {len(chunks)}")
print(f"📊 Expansão de chunking: {len(chunks)/len(documents):.1f}x")

✂️  Total de chunks criados: 33
📊 Expansão de chunking: 2.2x


## 🧮 Passo 5: Criar Embeddings

Usamos o modelo `embeddinggemma` rodando localmente via Ollama.

In [ ]:
# Cria embeddings
embeddings = OllamaEmbeddings(
    model='embeddinggemma', 
    base_url=OLLAMA_BASE_URL
    )

print("🧠 Modelo de embeddings: embeddinggemma")
print("✅ Embeddings configurados!")

🧠 Modelo de embeddings: embeddinggemma
✅ Embeddings configurados!


## 💾 Passo 6: Criar Vectorstore FAISS

Indexamos todos os chunks no FAISS para busca eficiente.

**Atenção:** Este passo pode demorar alguns minutos dependendo da quantidade de chunks!

In [ ]:
# Cria vectorstore
print("⏳ Criando índice FAISS (isso pode demorar alguns minutos...)\n")

vectorstore = FAISS.from_documents(
    documents=chunks, 
    embedding=embeddings
    )

print(f"\n✅ Vectorstore criado!")
print(f"📊 Total de vetores indexados: {vectorstore.index.ntotal}")
print(f"📐 Dimensões dos embeddings: {vectorstore.index.d}")

⏳ Criando índice FAISS (isso pode demorar alguns minutos...)


✅ Vectorstore criado!
📊 Total de vetores indexados: 33
📐 Dimensões dos embeddings: 768


## 🔍 Passo 7: Criar Retriever

### O que é um Retriever?

Um **retriever** é um componente do LangChain que encapsula a busca no vectorstore.

**Diferença:**

```python
# Abordagem manual:
docs = vectorstore.similarity_search(query, k=4)

# Usando Retriever:
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})
docs = retriever.invoke(query)  # Mesmo resultado!
```

### Por que usar Retriever?

✅ **Compatível com Chains:** Pode ser encadeado com `|`  
✅ **Interface padronizada:** Funciona com qualquer vectorstore  
✅ **Configurável:** Fácil trocar estratégias de busca  
✅ **Runnable:** Suporta invoke, stream, batch, etc.

**Configurações:**
- `search_kwargs={"k": 4}`: Retorna os 4 chunks mais similares
- Pode adicionar `score_threshold`, `fetch_k`, etc.

In [8]:
# Cria o retriever
retriever = vectorstore.as_retriever(
    search_type="similarity",  # Busca por similaridade
    search_kwargs={"k": 4}      # Retorna top 4 chunks
)

print("🔍 Retriever criado!")
print(f"📋 Configuração: busca os top-4 chunks mais similares")

🔍 Retriever criado!
📋 Configuração: busca os top-4 chunks mais similares


## 💬 Passo 8: Criar o Prompt Template

O mesmo template do notebook anterior, mas agora será usado dentro da chain.

In [9]:
# Template de prompt
template = """Use o seguinte contexto para responder à pergunta.
Se você não souber a resposta baseado no contexto, diga "Não encontrei essa informação nos documentos fornecidos."

Importante: Cite sempre a fonte (nome do arquivo e página) quando possível.

Contexto:
{context}

Pergunta: {question}

Resposta detalhada:"""

prompt = PromptTemplate(
    template=template,
    input_variables=['context', 'question']
)

print("📝 Prompt template criado!")

📝 Prompt template criado!


## 🤖 Passo 9: Criar o LLM

Modelo Ollama que gerará as respostas.

In [10]:
# Cria o LLM
llm = OllamaLLM(
    model='llama3.2:1b', 
    base_url=OLLAMA_BASE_URL
    )

print("🤖 LLM configurado: llama3.2:1b")

🤖 LLM configurado: llama3.2:1b


## 🔗 Passo 10: Criar a Chain RAG com LCEL

### 🎯 Este é o coração do notebook!

Vamos criar uma **chain** que automatiza todo o processo RAG usando o operador `|` (pipe).

### Anatomia da Chain:

```python
chain = (
    {
     "context": retriever | format_docs, 
     "question": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)
```

### O que cada parte faz?

#### 1️⃣ `{"context": ..., "question": ...}`
Cria um dicionário com as variáveis que o prompt precisa:

- **`"context": retriever | format_docs`**
  - Pega a pergunta → busca no vectorstore → formata os docs
  - `retriever`: Busca os 4 chunks mais similares
  - `|`: Passa os docs para a próxima função
  - `format_docs`: Junta os chunks com `\n\n`

- **`"question": RunnablePassthrough()`**
  - Passa a pergunta original sem modificar
  - `RunnablePassthrough()` = "deixa passar como está"

#### 2️⃣ `| prompt`
- Recebe o dicionário `{context: ..., question: ...}`
- Substitui os placeholders no template
- Retorna o prompt completo preenchido

#### 3️⃣ `| llm`
- Recebe o prompt completo
- Gera a resposta usando o LLM
- Retorna o texto bruto

#### 4️⃣ `| StrOutputParser()`
- Converte a saída do LLM em string limpa
- Remove metadados extras
- Retorna só o texto da resposta

### 🎭 Fluxo Completo:

```
"Quais são as formações do futebol?"
         ↓
    retriever → [doc1, doc2, doc3, doc4]
         ↓
    format_docs → "doc1\n\ndoc2\n\ndoc3\n\ndoc4"
         ↓
    {context: "...", question: "Quais são..."}
         ↓
    prompt → "Use o seguinte contexto... Pergunta: Quais são..."
         ↓
    llm → "As principais formações são..."
         ↓
    StrOutputParser → "As principais formações são..."
```

### ✨ Vantagens:

1. **Uma linha invoca tudo:** `chain.invoke("pergunta")`
2. **Composável:** Pode adicionar mais etapas facilmente
3. **Reutilizável:** Salva e usa em outros projetos
4. **Streaming:** `chain.stream()` para respostas progressivas
5. **Async:** `await chain.ainvoke()` para execução assíncrona

Vamos criar!

In [ ]:
# Função helper para formatar documentos
def format_docs(docs):
    """Formata lista de documentos em string única."""
    return "\n\n".join(doc.page_content for doc in docs)


# 🔗 Criando a Chain RAG com LCEL
rag_chain = (
    {
        "context": retriever | format_docs, 
        "question": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)

print("🔗 Chain RAG criada com sucesso!")
print("\n📋 Componentes da chain:")
print("  1️⃣  Retriever (busca documentos)")
print("  2️⃣  Format Docs (formata contexto)")
print("  3️⃣  Prompt Template (monta prompt)")
print("  4️⃣  LLM (gera resposta)")
print("  5️⃣  String Parser (formata saída)")
print("\n✅ Pronta para uso!")

## 🚀 Passo 11: Invocar a Chain

### Compare a simplicidade!

**Abordagem procedural (lab 3.3):**
```python
# 6 linhas de código
relevant_documents = vectorstore.similarity_search(query, k=4)
context = '\n\n'.join(doc.page_content for doc in relevant_documents)
question = query
prompt_result = prompt.invoke({'context': context, 'question': question})
response = llm.invoke(prompt_result)
print(response)
```

**Abordagem com Chain (agora):**
```python
# 1 linha de código!
response = rag_chain.invoke(query)
```

✨ **6 linhas → 1 linha!**

In [ ]:
# Pergunta de teste
query_futebol = "Quais são as principais formações táticas do futebol?"

print(f"❓ Pergunta: {query_futebol}\n")
print("⏳ Processando...\n")
print("=" * 80)

# 🎯 Uma linha invoca toda a chain!
response = rag_chain.invoke(query_futebol)

print("\n🤖 Resposta:")
print("=" * 80)
print(response)
print("=" * 80)

## 🔍 Passo 12: Inspecionar Documentos Recuperados

Vamos verificar quais documentos o retriever encontrou para essa pergunta.

In [ ]:
# Busca manual para inspeção
relevant_docs = retriever.invoke(query_futebol)

print(f"📚 Documentos recuperados: {len(relevant_docs)}\n")
print("=" * 80)

for i, doc in enumerate(relevant_docs, 1):
    print(f"\n📄 Documento {i}:")
    print(f"   Fonte: {doc.metadata.get('source', 'N/A')}")
    print(f"   Página: {doc.metadata.get('page', 'N/A')}")
    print(f"   Conteúdo (primeiros 150 chars):\n   {doc.page_content[:150]}...")
    print("-" * 80)

## 🧪 Passo 13: Testar com Múltiplas Perguntas

Vamos testar a chain com diferentes tipos de perguntas.

In [ ]:
# Lista de perguntas de teste
test_queries = [
    "Como fazer uma lasanha?",
    "Qual a diferença entre 4-4-2 e 4-3-3 no futebol?",
    "Quais são as regras do basquete?"
]

print("🧪 Testando múltiplas perguntas...\n")
print("=" * 80)

for i, query in enumerate(test_queries, 1):
    print(f"\n❓ Pergunta {i}: {query}")
    print("-" * 80)
    
    # Invoca a chain
    response = rag_chain.invoke(query)
    
    print(f"🤖 Resposta:\n{response}")
    print("=" * 80)

## 🌊 Passo 14: Streaming de Respostas (Bonus!)

Uma das grandes vantagens de usar chains é o suporte nativo a **streaming**!

### O que é Streaming?

Ao invés de esperar a resposta completa, você recebe **palavra por palavra** em tempo real.

**Benefícios:**
- 🚀 **UX melhor:** Usuário vê progresso imediato
- ⏱️ **Latência percebida:** Parece mais rápido
- 💬 **Chat-like:** Experiência tipo ChatGPT

**Uso:**
```python
for chunk in rag_chain.stream(query):
    print(chunk, end="", flush=True)
```

In [ ]:
# Teste com streaming
query_stream = "Quais são as principais formações táticas do futebol?"

print(f"❓ Pergunta: {query_stream}\n")
print("🌊 Resposta (streaming):")
print("=" * 80)

# Stream da resposta palavra por palavra
for chunk in rag_chain.stream(query_stream):
    print(chunk, end="", flush=True)

print("\n" + "=" * 80)
print("\n✅ Streaming concluído!")

## 📊 Passo 15: Análise do Sistema

Estatísticas sobre nosso sistema RAG com chains.

In [ ]:
# Estatísticas
print("📊 Estatísticas do Sistema RAG com Chains\n")
print("=" * 80)

print(f"\n📚 Dados:")
print(f"   Total de PDFs: {len(pdf_paths)}")
print(f"   Total de páginas: {len(documents)}")
print(f"   Total de chunks: {len(chunks)}")
print(f"   Expansão de chunking: {len(chunks)/len(documents):.1f}x")

# Tamanho dos chunks
tamanhos = [len(c.page_content) for c in chunks]
print(f"\n📏 Chunks:")
print(f"   Tamanho médio: {sum(tamanhos)/len(tamanhos):.0f} caracteres")
print(f"   Tamanho mínimo: {min(tamanhos)} caracteres")
print(f"   Tamanho máximo: {max(tamanhos)} caracteres")

# Vectorstore
print(f"\n🗄️  Vectorstore FAISS:")
print(f"   Dimensões dos embeddings: {vectorstore.index.d}")
print(f"   Total de vetores: {vectorstore.index.ntotal}")

# Chain
print(f"\n🔗 Chain RAG:")
print(f"   Componentes: 5 (retriever → format → prompt → llm → parser)")
print(f"   Suporte a streaming: ✅ Sim")
print(f"   Suporte a batch: ✅ Sim")
print(f"   Suporte a async: ✅ Sim")

print("\n" + "=" * 80)

## 🎓 Resumo: O que Aprendemos?

### ✅ Conceitos Principais

1. **LangChain Expression Language (LCEL)**
   - Sintaxe de pipeline usando `|`
   - Composição de componentes reutilizáveis
   - Padrão moderno para RAG

2. **Retriever**
   - Abstração sobre vectorstore
   - Interface `Runnable` compatível com chains
   - Configurável e padronizado

3. **Chain RAG**
   - Pipeline automático: retrieval → format → prompt → llm
   - Uma linha substitui 6+ linhas de código procedural
   - Suporte nativo a streaming, batch, async

4. **RunnablePassthrough**
   - Passa dados através da chain sem modificar
   - Útil para manter variáveis originais

5. **StrOutputParser**
   - Converte saída do LLM em string limpa
   - Remove metadados extras

### 🆚 Comparação: Procedural vs Chain

| Aspecto | Procedural (lab 3.3) | Chain (este lab) |
|---------|---------------------|------------------|
| **Linhas de código** | ~6 linhas | 1 linha |
| **Legibilidade** | Verboso | Conciso |
| **Streaming** | Manual | Nativo (`stream()`) |
| **Reutilização** | Difícil | Fácil (salva chain) |
| **Manutenção** | Repetitivo | DRY (Don't Repeat) |
| **Produção** | Não recomendado | Padrão da indústria |

### 🎯 Quando Usar Cada Abordagem?

**Procedural (lab 3.3):**
- ✅ Aprendizado e didática
- ✅ Debugging detalhado
- ✅ Entender cada etapa do RAG

**Chain (este lab):**
- ✅ Produção
- ✅ Projetos reais
- ✅ Código limpo e manutenível
- ✅ Performance e escalabilidade

### 🚀 Próximos Passos

1. **Experimente diferentes retrievers:**
   - `search_type="mmr"` (Maximum Marginal Relevance)
   - `search_type="similarity_score_threshold"`

2. **Adicione mais componentes à chain:**
   - Re-ranking de documentos
   - Filtros de metadados
   - Fallbacks e error handling

3. **Explore recursos avançados:**
   - `chain.batch([query1, query2])`: Processar múltiplas queries
   - `await chain.ainvoke(query)`: Execução assíncrona
   - `chain.with_config(...)`: Configuração dinâmica

4. **Otimize para produção:**
   - Cache de embeddings
   - Monitoramento e logging
   - Rate limiting

### 💡 Dica Final

**Chain = Composabilidade**

```python
# Você pode criar chains complexas facilmente:
chain = (
    preprocessor
    | retriever
    | reranker
    | prompt
    | llm
    | postprocessor
    | validator
)
```

Cada etapa é um componente reutilizável que pode ser testado isoladamente!

---

🎉 **Parabéns!** Você dominou RAG com Chains usando LCEL!